### Checking physical validity of simulations
Michael Shirts, CU Boulder
OSU MC/MD Summer Workshop, July 2022

In [ ]:
# enable plotting in notebook
%matplotlib notebook

# change shell to the correct version
%set_env SHELL=/bin/bash

## Checking whether a system of water molecules is physically consistent using GROMACS input files

### Preparing checks and loading data

Start by importing the `physical_validation` package.
Please refer to http://physical-validation.readthedocs.io/ for the full documentation.

In [ ]:
import physical_validation as pv

Create a GROMACS parser object, which needs the location of the GROMACS executable and the location of the topology include folder as inputs. Here, we assume that `gmx` is in the PATH, and that the topology folder is in its standard location. Change this if your local installation differs from this.

In [ ]:
gmx = pv.data.GromacsParser(exe='gmx_mpi',includepath='/opt/conda/share/gromacs/top/')

We'll test simulations ran with two thermostats:
`vr` stands for velocity-rescale, `be` for Berendsen thermostat.

In [ ]:
algos = ['vr', 'be']

We'll test simulations performed in two ensembles: NVT and NPT. 
Note that in NPT, the `vr` thermostat was complemented with a Parinello-Rahman barostat, while the `be` thermostat was complemented with a Berendsen barostat.

In [ ]:
ensembles = ['NVT', 'NPT']

The number of simulations performed in each ensemble:

* We need 2 simulations for NVT: Simulations at $T$ and $T+\Delta T$

* We use 4 simulations for NPT: $(T, P)$, $(T+\Delta T, P)$, $(T, P+\Delta P)$, $(T+\Delta T, P+\Delta P)$

There are actually three tests we can perform with NPT simulations.  

* Does the _volume_ (V) distribution change correctly when we vary the pressure $P$?
* Does the _enthalpy_ $(U+PV)$ distribution change correctly when we vary the $T$? 
* Does the 2D distribution of $U$ and $V$ change correctly when we change both $T$ and $P$? 


In [ ]:
sims = {'NVT': 2, 'NPT': 4}

The directories the data is stored are of the form:

In [ ]:
d = "md_vr_NVT_1"

Prepare a list we will store the parsed data objects in:

In [ ]:
simulation_data = list()

## Run checks

Now that we have prepared the setup variables, we will perform the checks. We will start with simulation data generated with the velocity-rescale thermostat of Bussi et al., and we will perform various checks of constant-volume ($NVT$) simulations.

Read in the simulation results using the GROMACS parser. This uses the `mdp` parameter file and the `top` topology file to gather information about the system and the simulation settings, and read the results from the `edr` file (trajectory of energy / volume / pressure / . . . ) and the `gro` file (position snapshot - used to read the box volume in $NVT$)

In [ ]:
for n in range(1,3):
    d = "systems/water/md_NVT_vr_"+str(n)
    simulation_data.append(gmx.get_simulation_data(
        mdp=d + "/mdout.mdp",
        edr=d + "/system.edr",
        gro=d + "/system.gro",
        top="systems/water/top/system.top",
    ))

## Test the kinetic energy distribution of the simulation results.

The first input is the simulation results we just read in, `strict` determines whether we test the full distribution (True) or only determine the mean and the variance of the distribution (False), `verbosity` sets the level of detail of the output  (with verbosity=0 being quiet and verbosity=3 being the most chatty), and the filename is being used to plot the resulting distribution for visual inspection (which we are also sending to the notebook).

Notice that we are doing a little bit of equilibration detection and subsampling here, as discussed earlier!

In [ ]:
print('==> Kinetic energy test of NVT simulations with velocity rescale simulation ')
print('==> Simulation 1 ')
pv.kinetic_energy.distribution(simulation_data[0], strict=True, verbosity=2)
print('==> Simulation 2 ')
pv.kinetic_energy.distribution(simulation_data[1], strict=True, verbosity=2)

Let's also do the non-strict test, as that can be a little more physically informative; we can see if the mean and standard deviation of the kinetic energy distributions are consistent with the temperature.

In [ ]:
pv.kinetic_energy.distribution(simulation_data[0], strict=False)
pv.kinetic_energy.distribution(simulation_data[1], strict=False)

We see that the simulations indeed have the correct temperatures, and the $T_{\mu}$ and $T_{\sigma}$ are consistent with each other. 

Just to be sure, let's look at things visually.  We can send the data to the screen, and we can specify a filename to save the picture to, as well. 

In [ ]:
pv.kinetic_energy.distribution(simulation_data[0], strict=False, verbosity=2,filename="vr_NVT_ke_0",screen=True)
pv.kinetic_energy.distribution(simulation_data[1], strict=False, verbosity=2,filename="vr_NVT_ke_1",screen=True)

## Hands on exercise

Check the kinetic energy distributions of the Berendsen weak-coupling thermostat (stored in directories with `be` instead of `vr` and see what you find!

## Next: Ensemble validation of NPT simulations

Let's compare the two simulations:

In [ ]:
pv.ensemble.check(simulation_data[0], simulation_data[1], verbosity=2, filename='vr_NVT_ensemble', screen=True)

## Exercise: 
    
 Does the Berendsen thermostat obey the correct distribution of energies? Look for data in the `br` directories. 
 

### NPT validation

There are three types of NPT validation we could do: Is the enthalpy distribution correct, is the volume distribution correct, and is the joint distribution of $(U,V)$ correct?.  To do that, we need four simulations. 
 - directories ending in `_1`: $T=300K$, $P=1$ atm
 - directories ending in `_2`: $T=308K$, $P=300$ atm
 - directories ending in `_3`: $T=300K$, $P=300$ atm
 - directories ending in `_4`: $T=308K$, $P=308$ atm


In [ ]:
#rezero the simulation data files
simulation_data=list()

In [ ]:
for n in range(1,5):
    d = "systems/water/md_NPT_vr_"+str(n)
    simulation_data.append(gmx.get_simulation_data(
        mdp=d + "/mdout.mdp",
        edr=d + "/system.edr",
        gro=d + "/system.gro",
        top="systems/water/top/system.top",
    ))

Note that the code automatically checks what ensemble the data was simulated in.  

In [ ]:
pv.ensemble.check(simulation_data[0], simulation_data[1], verbosity=2, filename='vr_NPT_ensemble', screen=True)

In [ ]:
pv.ensemble.check(simulation_data[0], simulation_data[2], verbosity=2, filename='vr_NPT_ensemble', screen=True)

In [ ]:
pv.ensemble.check(simulation_data[0], simulation_data[3], verbosity=2)

## Doing the same checks with just flat files of numbers.  

That was done with GROMACS simulations. But maybe we don't want to do GROMACS simulations!

In [ ]:
parser = pv.data.FlatfileParser()

We will need to manually enter information that was read automatically off of the GROMACS files. 

Our test system consists of 900 H$_2$O molecules whose bonds are fully constrained. Also, during the simulation, we kept the translation of the center of mass to zero, so we need to reduce the number of degrees of freedom by 3.

In [ ]:
system = pv.data.SystemData(
    natoms=900*3,
    nconstraints=900*3,
    ndof_reduction_tra=3,
    ndof_reduction_rot=0
)

The physical validation tests need some information on the units that were used in the simulation. While the strings are only used for output, the conversion respective to GROMACS units are relevant for the calculations. Please see the documentation for more information.

In [ ]:
units = pv.data.UnitData(
    kb=8.314462435405199e-3,
    energy_str='kJ/mol',
    energy_conversion=1.0,
    length_str='nm',
    length_conversion=1.0,
    volume_str='nm^3',
    volume_conversion=1.0,
    temperature_str='K',
    temperature_conversion=1.0,
    pressure_str='bar',
    pressure_conversion=1.0,
    time_str='ps',
    time_conversion=1.0
)

For the flat file example, we will only look at the simulations performed under NVT conditions using the velocity-rescale thermostat. There are two simulations: One ran at 300K, and one ran at 308K.

In [ ]:
ensemble_1 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=900*3,
    volume=3.01125**3,
    temperature=300
)
ensemble_2 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=900*3,
    volume=3.01125**3,
    temperature=308
)
dir_1 = 'systems/water/md_NVT_vr_1'
dir_2 = 'systems/water/md_NVT_vr_2'

We can now read in our flat data files (1-dimensional ascii files containing energy trajectories), and create a simulation result representation usable by the physical validation tests.

In [ ]:
result_1 = parser.get_simulation_data(
    units=units, ensemble=ensemble_1, system=system,
    kinetic_ene_file=dir_1 + '/kinetic_energy.dat',
    potential_ene_file=dir_1 + '/potential_energy.dat',
    total_ene_file=dir_1 + '/total_energy.dat'
)
result_2 = parser.get_simulation_data(
    units=units, ensemble=ensemble_2, system=system,
    kinetic_ene_file=dir_2 + '/kinetic_energy.dat',
    potential_ene_file=dir_2 + '/potential_energy.dat',
    total_ene_file=dir_2 + '/total_energy.dat'
)

### Now run the tests!

In [ ]:
print('==> Kinetic energy test of simulation ' + dir_1)
pv.kinetic_energy.distribution(result_1, strict=False, verbosity=2,
                               filename='ke_flat_vr_NVT_1', screen=True)
print('==> Kinetic energy test of simulation ' + dir_2)
pv.kinetic_energy.distribution(result_2, strict=False, verbosity=2,
                               filename='ke_flat_vr_NVT_2', screen=True)

In [ ]:
print('==> Potential energy test')
pv.ensemble.check(result_1, result_2,
                  verbosity=2, filename='pe_flat_vr_NVT', screen=True)

## Checking equipartition of velocities

Load in the data.  Note that we are now loading a `.trr` file, which contains the velocities.  You would need to make sure you tell gromacs to output velocities!

We have a set of directories with four sets of simulation.
* `vr` stands for the data generated with the velocity-rescale thermostat.
* `be` stands for the data generated using the Berendsen weak-coupling thermostat.
* `_1` denotes a simulation using a single thermostat for the entire system.
* `_2` denotes a simulation using two separate thermostats, one for the protein and one for the solvent.

Note that the topology and the trajectory have been modified to only contain the protein, as we are only interested in the equiparition of the solute, not the solvent. This reduces file size and execution time considerably.

In [ ]:
results_protein = gmx.get_simulation_data(mdp='systems/trp-cage/vr_1/protein.mdp',
                                              top='systems/trp-cage/vr_1/trp-cage.top',
                                              edr='systems/trp-cage/vr_1/run.edr',
                                              trr='systems/trp-cage/vr_1/protein.trr')       

One minor change we need to do because we removed the water. In the simulations, the center of mass was artificially kept immobile to avoid the build up of numerical errors, effectively reducing the number of translational degrees of freedom of the system by 3. As we are only looking at the protein here, we're using a mass-dependent fraction of these three degrees of freedom (weight of protein: 2170.4375 amu; total weight of the system: 96390.4017 amu).

If we were doing this with the full system (solvent + water), then we could just remove 3 degrees of freedom. 

In [ ]:
results_protein.system.ndof_reduction_tra *= 2170.4375 / 96390.4017

In [ ]:
pv.kinetic_energy.equipartition(results_protein,strict=False, filename='equipartition_vr_1', screen=True,
                                    verbosity=3)

Now let's look at Berendsen weak coupling!

In [ ]:
results_protein = gmx.get_simulation_data(mdp='systems/trp-cage/be_1/protein.mdp',
                                              top='systems/trp-cage/be_1/trp-cage.top',
                                              edr='systems/trp-cage/be_1/run.edr',
                                              trr='systems/trp-cage/be_1/protein.trr')  

In [ ]:
results_protein.system.ndof_reduction_tra *= 2170.4375 / 96390.4017

In [ ]:
pv.kinetic_energy.equipartition(results_protein,strict=False, filename='equipartition_be_1', screen=True,
                                    verbosity=3)

## Exercise: 
Now see what happens when we use two different thermostatting groups, one each for the solvent and the protein, with both the velocity rescale thermostat (`vr`) and the Berendsen weak-coupling thermostat (`be`).